In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
# from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [ ]:
import numpy as np

#usare Scikit-learn es una de las bibliotecas más populares para el aprendizaje automático en Python. La función train_test_split se utiliza
#para dividir un conjunto de datos en dos subconjuntos: uno para entrenamiento y otro para pruebas
from sklearn.model_selection import train_test_split


# La entrada es de 784 representando mis variables X
input_layer_size  = 56

# 10 etiquetas, de 1 a 10 (tomar en cuenta que se asigna "0" a la etiqueta 10)
num_labels = 2

#cargo mi dataset entrenar y para evaluar algoritmos de reconocimiento de digitos escritos a mano
data = np.loadtxt('/content/drive/MyDrive/primelParcial/preparado_1.txt', delimiter=';')

# Dividir los datos en características (X) y etiquetas (y)
X = data[:, 1:]
y = data[:, 55]
m = y.size



# declarando mis variables para mi division de (80% para entrenamiento, 20% para pruebas)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#imprimo mi 80% de datos de entrenamiento de mi dataset tanto mis variables x mi y
print("Tamaño de datos de entrenamiento para X:", X_train.shape)
print("Tamaño de datos de entrenamiento para y:", y_train.shape)


#imprimo mi 20% de datos para la prueba de mi dataset tanto mis variables x mi y
print("Tamaño de datos de prueba para X:", X_test.shape)
print("Tamaño de datos de prueba para y:", y_test.shape)



Tamaño de datos de entrenamiento para X: (46876, 55)
Tamaño de datos de entrenamiento para y: (46876,)
Tamaño de datos de prueba para X: (11720, 55)
Tamaño de datos de prueba para y: (11720,)


In [ ]:
print(data.shape)
print(y)

(58596, 56)
[0. 0. 0. ... 1. 1. 1.]


In [ ]:
#haora normalizo mis los valores de mis X  para asegurarme que esten en una misma escala
# tanto los de prueba como los de entrenamiento
def featureNormalize(X_train, X_test):
    mu = np.mean(X_train, axis=0)
    sigma = np.std(X_train, axis=0)

    # agrego este sigma el caso de desviación estándar cero por si hay division entre 0
    sigma[sigma == 0] = 1e-8

    #aplico normalizacion
    X_train_normalized = (X_train - mu) / sigma
    X_test_normalized = (X_test - mu) / sigma

    return X_train_normalized, X_test_normalized, mu, sigma

#llamo a mi  función para normalizar los datos
X_train_normalized, X_test_normalized, mu, sigma = featureNormalize(X_train, X_test)

# Imprimo el tamaño de los conjuntos de entrenamiento y prueba normalizados
print("Tamaño de datos de entrenamiento normalizados para X:", X_train_normalized.shape)
print("Tamaño de datos de prueba normalizados para X:", X_test_normalized.shape)



Tamaño de datos de entrenamiento normalizados para X: (46876, 55)
Tamaño de datos de prueba normalizados para X: (11720, 55)


In [ ]:
# asigno a una varaible nueva pasandole las variables normalizadas y las columnas
m_train, n_train = X_train_normalized.shape
m_test, n_test = X_test_normalized.shape

X_train_norm = X_train_normalized
X_test_norm =X_test_normalized


In [ ]:
print("Conjunto de entrenamiento normalizado:")
print(X_train_norm)
print("Conjunto de prueba normalizado:")
print(X_test_norm)


Conjunto de entrenamiento normalizado:
[[ 0.48678508 -0.7469441   0.         ... -0.02958739  0.01222098
   1.00299108]
 [ 0.86357108  2.05508743  0.         ... -0.02958739  0.01222098
   1.00299108]
 [-1.39714495 -0.35122079  0.         ... -0.02958739  0.01222098
  -0.99701784]
 ...
 [-1.02035895 -0.35484754  0.         ... -0.02958739  0.01222098
  -0.99701784]
 [ 1.9939291  -0.39987201  0.         ... -0.02958739  0.01222098
  -0.99701784]
 [ 0.48678508  2.23063318  0.         ... -0.02958739  0.01222098
   1.00299108]]
Conjunto de prueba normalizado:
[[ 0.10999907 -0.44158365  0.         ... -0.02958739  0.01222098
   1.00299108]
 [-1.02035895 -0.34227028  0.         ... -0.02958739  0.01222098
  -0.99701784]
 [ 0.48678508 -0.43109445  0.         ... -0.02958739  0.01222098
   1.00299108]
 ...
 [ 0.86357108  2.2130451   0.         ... -0.02958739  0.01222098
   1.00299108]
 [ 0.48678508  2.09796883  0.         ... -0.02958739  0.01222098
   1.00299108]
 [ 0.48678508 -0.74694436  

In [ ]:
# Selecciona aleatoriamente 100 puntos de datos para mostrar
# y la guardare el sel
rand_indices = np.random.choice(m, 100, replace=False)
sel = X[rand_indices, :]

# displayData(sel)

In [ ]:
# usare la sigmoide ´para  tomar un valor real como entrada y produce un valor en el rango de 0 a 1 como salida
def sigmoid(z):

    return 1.0 / (1.0 + np.exp(-z))

In [ ]:
#Usare una función de costo y su gradiente para un problema de regresión logística regularizada.
#para encontrar los parámetros theta que minimizan la función de costo.

def lrCostFunction(theta, X, y, lambda_):

    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

In [ ]:
# un ejemplo de calculo del costo
theta_initial = np.zeros(X_train_normalized.shape[1])

# Definir el valor de lambda
lambda_ = 0.1

# Calcular el costo y el gradiente utilizando la función de costo
costo, _ = lrCostFunction(theta_initial, X_train_normalized, y_train, lambda_)

print("Costo de entrenamiento:", costo)


Costo de entrenamiento: 0.6931471805599467


In [ ]:
#usare scipy  para utilizar sus funcionalidades de optimización numérica en Python.
#entrenare con  "One-vs-All",para que me devuelba loss parámetros óptimos del modelo para cada clase.
from scipy import optimize

def oneVsAll(X, y, num_labels, lambda_):
    # Algunas variables útiles
    m, n = X.shape

    # Inicializar matriz de parámetros theta
    all_theta = np.zeros((num_labels, n + 1))

    # Agregar una columna de unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    # Optimizar los parámetros theta para cada clase
    for c in range(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}  # Opciones para el optimizador
        # Minimizar la función de costo regularizada utilizando el método de conjugado de gradiente
        res = optimize.minimize(lrCostFunction, initial_theta, (X, (y == c), lambda_), jac=True, method='CG', options=options)
        # Asignar los parámetros theta óptimos a la fila correspondiente en all_theta
        all_theta[c] = res.x

    return all_theta


In [ ]:
#entreno un clasificador usando uno vs todos e imprimo mis valores de mis clases

lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)

<ipython-input-26-325fe13cf96f>:4: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
<ipython-input-27-0fae340bc49a>:21: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-26-325fe13cf96f>:4: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
<ipython-input-27-0fae340bc49a>:21: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-26-325fe13cf96f>:4: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
<ipython-input-27-0fae340bc49a>:21: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-26-325fe13cf96f>:4: RuntimeWarn

(10, 56)


<ipython-input-26-325fe13cf96f>:4: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
<ipython-input-27-0fae340bc49a>:21: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))


In [ ]:
#tomare los valores óptimos del modelo (all_theta) y un conjunto de datos de entrada (X), y
#predice las etiquetas de clase para cada muestra en X utilizando el enfoque de "One-vs-All" en la regresión logística

def predictOneVsAll(all_theta, X):

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [ ]:
# Usando mi 20% de datos de prueba
print(X_test_norm.shape)

# realizare  predicciones en el conjunto de prueba usando el 20%
pred_test = predictOneVsAll(all_theta, X_test_norm)

#  la precisión en el conjunto de prueba usando mi 20% de y   de prueba
accuracy_test = np.mean(pred_test == y_test) * 100

# Imprimir la precisión del conjunto de prueba
print('Precisión en el conjunto de prueba: {:.2f}%'.format(accuracy_test))

# al igual que el ejemplo del ing seleccionare un subconjunto de datos de prueba para mostrar un ejemplo
X_prueba_subconjunto = X_test_norm[10:150, :]
y_prueba_subconjunto = y_test[10:150]

# hoara agregare una columna de unos a la matriz X_prueba_subconjunto para realizar mis prediccion
X_prueba_subconjunto = np.concatenate([np.ones((X_prueba_subconjunto.shape[0], 1)), X_prueba_subconjunto], axis=1)

# realizando las predicciones en el subconjunto de datos de prueba
pred_prueba_subconjunto = np.argmax(sigmoid(X_prueba_subconjunto.dot(all_theta.T)), axis=1)

# Imprimo las predicciones y las etiquetas del mis 20% de datos de prueba aver si sale bien :V
print("Predicciones del subconjunto de datos de prueba:")
print(pred_prueba_subconjunto)
print("Etiquetas reales del subconjunto de datos de prueba:")
print(y_prueba_subconjunto)


(11720, 55)
Precisión en el conjunto de prueba: 42.72%
Predicciones del subconjunto de datos de prueba:
[0 1 0 1 2 2 1 2 2 2 2 0 2 0 0 2 0 1 1 0 1 1 0 0 2 2 0 2 0 0 2 1 1 2 2 1 2
 2 2 0 0 1 2 1 0 0 2 0 2 2 2 2 2 0 0 1 2 1 2 1 2 0 1 2 2 0 0 2 2 0 1 2 2 1
 1 2 1 1 2 0 1 0 2 0 1 2 2 1 1 2 0 2 0 0 1 2 2 2 2 2 0 2 1 0 2 2 1 2 1 0 1
 2 1 1 2 1 2 1 1 1 2 2 1 0 2 0 2 2 0 0 2 1 2 1 1 0 0 2 2 2]
Etiquetas reales del subconjunto de datos de prueba:
[0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0.
 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1.
 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1.
 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1.
 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1.]


¡Claro! La técnica One-vs-All (también conocida como One-vs-Rest) es una técnica de clasificación que se puede aplicar tanto a problemas de clasificación binaria como a problemas de clasificación multiclase.

En un problema de clasificación binaria, como el que tienes (por ejemplo, determinar si un dato tiene o no tiene malware), la técnica One-vs-All aún puede ser utilizada. La idea es entrenar múltiples clasificadores binarios, donde cada clasificador intenta distinguir una clase específica de las otras.

En tu caso, dado que tienes un problema binario (malware o no malware), solo necesitas un clasificador. Sin embargo, la técnica One-vs-All sigue siendo aplicable conceptualmente. Puedes pensar en ello como entrenar un clasificador que distingue "malware" de "no malware" (es decir, un clasificador que busca la clase "1" frente a la clase "0"), lo que esencialmente es un problema de clasificación binaria.

Por lo tanto, aunque en la práctica solo estás resolviendo un problema de clasificación binaria, puedes utilizar la técnica One-vs-All para conceptualmente encapsularlo y aplicarla de manera coherente en la implementación.

Espero que esto aclare por qué se puede aplicar la técnica One-vs-All en un problema de clasificación binaria como el tuyo. Si tienes alguna otra pregunta o necesitas más aclaraciones, no dudes en preguntar.